In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

import os
train_data_set = pd.DataFrame(columns=['filename' , 'isSmoker'])
somker_Counter = 0
non_smoker_counter = 0 
all_count = 0
for dirname, _, filenames in os.walk('/kaggle/input/smoking/Training/Training'):
    for filename in filenames:
        all_count+=1
        if filename.find('notsmoking') !=-1 :
            train_data_set.loc[len(train_data_set)] = {'filename':filename ,'isSmoker':0}
            non_smoker_counter +=1
            
        else :
            somker_Counter +=1
            train_data_set.loc[len(train_data_set)] = {'filename':filename ,'isSmoker':1}
        

In [3]:
vaildtion_data_set = pd.DataFrame(columns=['filename' , 'isSmoker'])
for dirname, _, filenames in os.walk('/kaggle/input/smoking/Validation/Validation'):
    for filename in filenames:
        if filename.find('notsmoking') !=-1 :
            vaildtion_data_set.loc[len(vaildtion_data_set)] = {'filename':filename ,'isSmoker':0}
            
        else :
            vaildtion_data_set.loc[len(vaildtion_data_set)] = {'filename':filename ,'isSmoker':1}

In [4]:
train_data_set

,filename,isSmoker
0,smoking_0175.jpg,1
1,smoking_0480.jpg,1
2,notsmoking_0267.jpg,0
3,notsmoking_0037.jpg,0
4,notsmoking_0379.jpg,0
...,...,...
711,notsmoking_0008.jpg,0
712,notsmoking_0039.jpg,0
713,notsmoking_0492.jpg,0
714,smoking_0278.jpg,1


In [5]:
vaildtion_data_set

,filename,isSmoker
0,smoking_0514.jpg,1
1,smoking_0036.jpg,1
2,smoking_0043.jpg,1
3,smoking_0362.jpg,1
4,notsmoking_0145.jpg,0
...,...,...
175,smoking_0516.jpg,1
176,notsmoking_0517.jpg,0
177,smoking_0047.jpg,1
178,smoking_0161.jpg,1


In [15]:
print('the number of non_smoker in training data set is '+str(non_smoker_counter))
print('the number of smoker in training data set is '+str(somker_Counter))
print('the number of all records is '+str(all_count))



the number of non_smoker in training data set is 358
the number of smoker in training data set is 358
the number of all records is 716


In [16]:
test_data_set = pd.DataFrame(columns=['filename' , 'isSmoker'])
somker_Counter = 0
non_smoker_counter = 0 
all_count = 0
for dirname, _, filenames in os.walk('/kaggle/input/smoking/Testing/Testing'):
    for filename in filenames:
        all_count+=1
        if filename.find('notsmoking') !=-1 :
            test_data_set.loc[len(test_data_set)] = {'filename':filename ,'isSmoker':0}
            non_smoker_counter +=1
            
        else :
            somker_Counter +=1
            test_data_set.loc[len(test_data_set)] = {'filename':filename ,'isSmoker':1}

In [17]:
print('the number of non_smoker in testing data set is '+str(non_smoker_counter))
print('the number of smoker in testing data set is '+str(somker_Counter))
print('the number of all records is '+str(all_count))


the number of non_smoker in testing data set is 112
the number of smoker in testing data set is 112
the number of all records is 224


In [18]:
train_Image_genritor = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255 ,rotation_range= 0.3 , shear_range= 0.2 , horizontal_flip= 0.2 , zoom_range= 0.3 )
train_Data = train_Image_genritor.flow_from_dataframe(train_data_set ,directory= '/kaggle/input/smoking/Training/Training',target_size = (256,256) ,y_col ='isSmoker',x_col = 'filename' ,class_mode='raw')

Found 716 validated image filenames.


In [19]:
vaildtion_genritor = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255)
vaildate_data = vaildtion_genritor.flow_from_dataframe(vaildtion_data_set , directory='/kaggle/input/smoking/Validation/Validation' , target_size = (256 ,256) , class_mode='raw' ,y_col ='isSmoker',x_col = 'filename')

Found 180 validated image filenames.


In [20]:
test_genritor = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255)
test_data = vaildtion_genritor.flow_from_dataframe(test_data_set , directory='/kaggle/input/smoking/Testing/Testing' , target_size = (256 ,256) , class_mode='raw' ,y_col ='isSmoker',x_col = 'filename')


Found 224 validated image filenames.


In [21]:
vaildate_data.image_shape

(256, 256, 3)

In [22]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32 , 3 ,activation='relu' , input_shape = (256 ,256,3)) ,
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.20),
    
    tf.keras.layers.Conv2D(64 , 3 ,activation='relu') ,
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.20),
    
    tf.keras.layers.Conv2D(126 , 3, activation='relu') , 
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.20),
    
    tf.keras.layers.Conv2D(258 , 3 , activation= 'relu') ,
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.20),
    
    tf.keras.layers.Conv2D(512 , 3, activation= 'relu') ,
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.20),
    
    tf.keras.layers.Flatten(), 
    
    tf.keras.layers.Dense(1000,activation='relu') ,
    tf.keras.layers.Dense(500,activation='relu') ,
    tf.keras.layers.Dense(200,activation='relu') ,
    tf.keras.layers.Dense(1,activation='sigmoid') ,
    

    
    
])

In [23]:
model.compile(optimizer= tf.keras.optimizers.SGD(learning_rate = 1e-2) , loss = tf.keras.losses.BinaryCrossentropy() , 
            metrics= ['accuracy'])

In [25]:
model.fit(train_Data ,epochs=50 , validation_data= vaildate_data )

Epoch 1/50


2025-04-23 11:25:37.309039: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


23/23 [==============================] - 14s 462ms/step - loss: 0.6877 - accuracy: 0.5209 - val_loss: 0.6885 - val_accuracy: 0.5000
Epoch 2/50
23/23 [==============================] - 11s 476ms/step - loss: 0.6752 - accuracy: 0.5084 - val_loss: 0.6851 - val_accuracy: 0.4944
Epoch 3/50
23/23 [==============================] - 11s 474ms/step - loss: 0.6666 - accuracy: 0.5670 - val_loss: 0.6808 - val_accuracy: 0.5278
Epoch 4/50
23/23 [==============================] - 11s 475ms/step - loss: 0.6593 - accuracy: 0.5573 - val_loss: 0.6767 - val_accuracy: 0.6500
Epoch 5/50
23/23 [==============================] - 11s 482ms/step - loss: 0.6522 - accuracy: 0.6075 - val_loss: 0.6714 - val_accuracy: 0.6667
Epoch 6/50
23/23 [==============================] - 11s 467ms/step - loss: 0.6436 - accuracy: 0.6313 - val_loss: 0.6717 - val_accuracy: 0.6944
Epoch 7/50
23/23 [==============================] - 11s 471ms/step - loss: 0.6506 - accuracy: 0.5950 - val_loss: 0.6653 - val_accuracy: 0.6944
Epoch 8/50

In [26]:
loss, accuracy = model.evaluate(test_data)
print(f'Test Accuracy: {accuracy:.2f}')

7/7 [==============================] - 2s 290ms/step - loss: 0.5928 - accuracy: 0.7188
Test Accuracy: 0.72
